In [18]:
from DataLoader import importar_ventas,importar_cluster_stores,leer_csv,agregar_fourier,preparar_test
from DataLoader import crear_archivo_kaggle
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import r2_score

In [21]:
def r2_metric(pred, dtrain):
    label = dtrain.get_label()
    
    loss = (pred - label) ** 2
    var = (label - np.mean(label)) ** 2
    
    r2 = 1 - sum(loss) / sum(var)
    return 'r2', r2, True

In [2]:
ventas=importar_ventas()

In [3]:
ventas

,date,demand,mean_price,std_price,max_price,min_price,cluster,store_cod,subgroup_cod,demand_lag_1,demand_lag_2,demand_lag_3,demand_lag_4,demand_lag_5,demand_lag_6,demand_lag_7,day,month,year
0,2021-01-10,1.0,319.510000,0.000000,319.51,319.51,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,1,2021
1,2021-01-13,8.0,241.460000,0.000000,241.46,241.46,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,13,1,2021
2,2021-01-18,4.0,250.180000,0.000000,250.18,250.18,0,0,0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,18,1,2021
3,2021-01-22,5.0,268.270000,0.000000,268.27,268.27,0,0,0,4.0,8.0,1.0,NaN,NaN,NaN,NaN,22,1,2021
4,2021-01-23,3.0,297.770000,0.000000,297.77,297.77,0,0,0,5.0,4.0,8.0,1.0,NaN,NaN,NaN,23,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206788,2023-12-15,2.0,27.320000,0.000000,27.32,27.32,8,156,73,5.0,6.0,7.0,5.0,3.0,6.0,2.0,15,12,2023
5206789,2023-12-16,15.0,29.456667,3.035845,33.75,27.31,8,156,73,2.0,5.0,6.0,7.0,5.0,3.0,6.0,16,12,2023
5206790,2023-12-21,3.0,26.450000,0.000000,26.45,26.45,8,156,73,15.0,2.0,5.0,6.0,7.0,5.0,3.0,21,12,2023
5206791,2023-12-26,2.0,26.530000,0.000000,26.53,26.53,8,156,73,3.0,15.0,2.0,5.0,6.0,7.0,5.0,26,12,2023


In [4]:
test=preparar_test("ids_test.csv")
#test

In [5]:
agregar_fourier(ventas,2)
agregar_fourier(test,2)

In [6]:

FEATURES=[
    #'mean_price', 
    #'cluster', 
    #'store_cod', 
    #'subgroup_cod',
    #'demand_lag_1', #'demand_lag_2', 'demand_lag_3', 
    #'demand_lag_4','demand_lag_5', 'demand_lag_6', 'demand_lag_7', 'day', 'month', 
    'year',
   'anual_sin_1', 
    'anual_cos_1',
   # 'mensual_sin_1', 
    #'mensual_cos_1',
   #'anual_sin_2', 
    #'anual_cos_2',
    'mensual_sin_2', 
    #'mensual_cos_2',
    "std_price",
    "max_price", 
   "min_price"
] #recordar que ya probamos con fourier 3 y 4 y fue un fracaso

TARGET=['demand']

test_interno=ventas.groupby(["subgroup_cod","store_cod"]).tail(7)
train=ventas
train_interno=ventas.drop(test_interno.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test_real=test[FEATURES]

X_train_interno=train_interno[FEATURES]
y_train_interno=train_interno[TARGET]

X_test_interno=test_interno[FEATURES]
y_test_interno=test_interno[TARGET]

# X_test_2023=test_2023[FEATURES]
# y_test_2023=test_2023[TARGET]

# X_train_2023=train_2023[FEATURES]
# y_train_2023=train_2023[TARGET]

In [28]:
reg_lgb = lgb.LGBMRegressor(
    n_estimators=600, #deberia parar con el early stopping
    learning_rate=0.3, 
    max_depth=8,
    objective='regression',
    # colsample_bytree=0.9,
    verbose=1  
)

def r2_score_lgb(y_true, y_pred):
    return 'r2', r2_score(y_true, y_pred), True 

reg_lgb.fit(
    X_train, y_train,
    eval_set=[(X_train_interno, y_train_interno)], 
    eval_metric=r2_score_lgb,
    callbacks=[lgb.early_stopping(stopping_rounds=50),lgb.log_evaluation(50) ]
)
                    

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038531 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1283
[LightGBM] [Info] Number of data points in the train set: 5206793, number of used features: 7
[LightGBM] [Info] Start training from score 4.936933
Training until validation scores don't improve for 50 rounds
[50]	valid_0's l2: 7.17124	valid_0's r2: 0.439956
[100]	valid_0's l2: 7.05429	valid_0's r2: 0.449089
[150]	valid_0's l2: 6.99904	valid_0's r2: 0.453404
[200]	valid_0's l2: 6.9527	valid_0's r2: 0.457023
[250]	valid_0's l2: 6.92311	valid_0's r2: 0.459334
[300]	valid_0's l2: 6.89995	valid_0's r2: 0.461142
[350]	valid_0's l2: 6.87955	valid_0's r2: 0.462736
[400]	valid_0's l2: 6.86306	valid_0's r2: 0.464023
[450]	valid_0's l2: 6.84551	valid_0's r2: 0.465394
[500]	valid_0's l2: 6.8314	valid_0's r2: 0.466496
[550]	valid_0's l2: 6.818

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,8
,learning_rate,0.3
,n_estimators,600
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [29]:
feature_importance = pd.DataFrame(
    reg_lgb.feature_importances_,
    columns=["importance"],
    index=reg_lgb.feature_name_
).sort_values(by="importance", ascending=False)

feature_importance

,importance
std_price,4112
max_price,3673
min_price,3622
anual_sin_1,2860
anual_cos_1,2120
year,1204
mensual_sin_2,409
